# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [46]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
import warnings

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [47]:
warnings.simplefilter("ignore", category=FutureWarning)

stocks = pd.read_csv("sp_500_stocks.csv")

errors = ['DISCA','HFC','VIAC','WLTW']

for i in errors:
    stocks = stocks.mask(stocks['Ticker']==i).dropna()

for i in stocks["Ticker"]:
    print(i)

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INF

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [48]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [49]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 81764390,
 'calculationPrice': 'tops',
 'change': -0.691,
 'changePercent': -0.0054,
 'close': 0,
 'closeSource': 'clioifaf',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 134.8,
 'iexAskSize': 124,
 'iexBidPrice': 134.06,
 'iexBidSize': 314,
 'iexClose': 134.859,
 'iexCloseTime': 1744636281782,
 'iexLastUpdated': 1679666948810,
 'iexMarketPercent': 0.01952387828631708,
 'iexOpen': 132.24,
 'iexOpenTime': 1677611619482,
 'iexRealtimePrice': 132.01,
 'iexRealtimeSize': 10,
 'iexVolume': 347134,
 'lastTradeTime': 1724319768312,
 'latestPrice': 132.185,
 'latestSource': 'IEX real time price',
 'latestTime': '10:46:45 AM',
 'latestUpdate': 1678188980666,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [50]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [51]:
my_columns = ["Ticker","Stock Price","Market cap","Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market cap,Number of Shares to Buy


In [52]:
final_dataframe.append(pd.Series([symbol,price,market_cap,'N/A'],index = my_columns),ignore_index = True)

,Ticker,Stock Price,Market cap,Number of Shares to Buy
0,AAPL,132.185,2154883489727,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [26]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:1]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],index = my_columns),ignore_index = True)
    
    

In [27]:
final_dataframe

,Ticker,Stock Price,Market cap,Number of Shares to Buy
0,A,151.71,44533191821,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [53]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [54]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], #stock, endpoint, metric
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market cap,Number of Shares to Buy
0,A,150.950,45537154559,N/A
1,AAL,12.870,8067607005,N/A
2,AAP,152.480,9113517528,N/A
3,AAPL,129.760,2078952503579,N/A
4,ABBV,169.767,291891152380,N/A
...,...,...,...,...
496,YUM,135.970,36742496366,N/A
497,ZBH,131.830,26996126935,N/A
498,ZBRA,252.680,13220421777,N/A
499,ZION,49.146,7344101770,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [92]:
portfolio = input("Enter the value of your Portfolio: ")

try:
    val = float(portfolio)
except ValueError:
    print("Not a number. Enter a number")
    portfolio = input("Enter the value of your Portfolio: ")
    val = float(portfolio)


Enter the value of your Portfolio: 1000000


In [93]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i,"Stock Price"])

final_dataframe

,Ticker,Stock Price,Market cap,Number of Shares to Buy
0,A,150.950,45537154559,13
1,AAL,12.870,8067607005,155
2,AAP,152.480,9113517528,13
3,AAPL,129.760,2078952503579,15
4,ABBV,169.767,291891152380,11
...,...,...,...,...
496,YUM,135.970,36742496366,14
497,ZBH,131.830,26996126935,15
498,ZBRA,252.680,13220421777,7
499,ZION,49.146,7344101770,40


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [105]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [106]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    
    {'font_color':font_color,
     'bg_color':background_color,
     'border':1,
     'align':'left'}
)

dollar_format = writer.book.add_format(
    
    {'num_format':'$0.00',
     'font_color':font_color,
     'bg_color':background_color,
     'border':1,
     'align':'left'}
)

integer_format = writer.book.add_format(
    {'num_format':'0',
     'font_color':font_color,
     'bg_color':background_color,
     'border':1,
     'align':'left'}
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [103]:
# writer.sheets["Recommended Trades"].write('A1','Ticker',string_format) #column, font-pixel,format
# writer.sheets["Recommended Trades"].write('B1','Stock Price',dollar_format)
# writer.sheets["Recommended Trades"].write('C1','Market Cap',dollar_format)
# writer.sheets["Recommended Trades"].write('D1','Number of Shares to Buy',integer_format)


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [107]:
column_format = {
    'A':['Ticker',string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Cap',dollar_format],
    'D':['Number of Shares to Buy',integer_format]
}

for column in column_format.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}',18,column_format[column][1])
    
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy: